In [1]:
import cv2
import tkinter as tk
import xml.etree.ElementTree as ET
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
import supervision as sv
import torch
import io
from PIL import Image, ImageTk
import tkinter.filedialog as filedialog
import uuid 



DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
MODEL_TYPE = "vit_h"
CHECKPOINT_PATH="sam_vit_h_4b8939.pth"

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH)

In [16]:
 # Para generar identificadores únicos

# Variable global para almacenar la imagen PhotoImage
photo = None

class App:
    def __init__(self, root):
        self.root = root
        self.root.title("Segmentación de objetos")
        self.photo = None
        self.image = None  # Añade la variable de imagen
        self.annotated_image = None  # Imagen segmentada con contornos
        self.objects = {}  # Diccionario para almacenar objetos

        # Configurar tamaño de la ventana
        self.root.geometry("800x600")  # Configura las dimensiones deseadas

        # Crear la ventana principal
        self.create_widgets()

    def create_widgets(self):
        # Botón para abrir un archivo de imagen
        open_button = tk.Button(self.root, text="Abrir Imagen", command=self.load_image)
        open_button.pack()

        # Botón para guardar coordenadas en un archivo XML
        save_button = tk.Button(self.root, text="Guardar Coordenadas", command=self.save_coordinates)
        save_button.pack()

        # Label para mostrar la imagen
        self.label = tk.Label(self.root)
        self.label.pack()

        # Configurar manejo de clics en la imagen
        self.label.bind("<Button-1>", self.handle_click)

    def load_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            # Cargar la imagen
            self.image = cv2.imread(file_path)  # Almacena la imagen en el atributo 'image'

            # Inicializar SegmentAnything
            sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH)   # Define sam adecuadamente con la configuración necesaria
            mask_generator = SamAutomaticMaskGenerator(sam)

            result = mask_generator.generate(self.image)
            mask_annotator = sv.MaskAnnotator()
            detections = sv.Detections.from_sam(result)
            self.annotated_image = mask_annotator.annotate(self.image.copy(), detections)  # Almacena la imagen segmentada original

            # Mostrar la imagen segmentada
            img = Image.fromarray(cv2.cvtColor(self.annotated_image, cv2.COLOR_BGR2RGB))
            self.photo = ImageTk.PhotoImage(img)

            # Verificar si self.photo se ha inicializado correctamente
            if self.photo:
                self.label.config(image=self.photo)
            else:
                print("Error: La imagen no se ha cargado correctamente.")

    def handle_click(self, event):
        x, y = event.x, event.y
        print("Coordenadas del clic:", x, y)
        color = self.get_pixel_color(x, y)  # Accede a 'self.image' en lugar de 'image'

        # Encuentra todas las coordenadas del mismo color
        connected_coordinates = self.find_connected_coordinates(x, y, color)

        # Genera un identificador único para el objeto
        object_id = str(uuid.uuid4())

        # Almacena las coordenadas en el diccionario de objetos con el identificador
        self.objects[object_id] = connected_coordinates

        # Dibuja un contorno alrededor de todas las coordenadas del mismo color en la imagen segmentada original
        for coord in connected_coordinates:
            cv2.circle(self.annotated_image, (coord[0], coord[1]), 3, (0, 255, 0), -1)  # Dibuja un círculo

        # Actualiza la imagen segmentada en la interfaz de usuario
        img = Image.fromarray(cv2.cvtColor(self.annotated_image, cv2.COLOR_BGR2RGB))
        self.photo = ImageTk.PhotoImage(img)
        self.label.config(image=self.photo)

    def get_pixel_color(self, x, y):
        b, g, r = self.image[y, x]  # Accede a 'self.image' en lugar de 'image'
        return (r, g, b)

    def find_connected_coordinates(self, x, y, target_color):
        color_threshold = 20  # Umbral de diferencia de color permitido

        # Inicializa el conjunto de coordenadas conectadas
        connected_coordinates = set()
        connected_coordinates.add((x, y))

        # Inicializa la lista de coordenadas para explorar
        to_explore = [(x, y)]

        while to_explore:
            current_x, current_y = to_explore.pop()

            # Verifica los píxeles vecinos
            neighbors = [(current_x - 1, current_y),
                         (current_x + 1, current_y),
                         (current_x, current_y - 1),
                         (current_x, current_y + 1)]

            for neighbor_x, neighbor_y in neighbors:
                if (0 <= neighbor_x < self.image.shape[1] and
                    0 <= neighbor_y < self.image.shape[0] and
                    self.get_pixel_color(neighbor_x, neighbor_y) == target_color and
                    (neighbor_x, neighbor_y) not in connected_coordinates):

                    connected_coordinates.add((neighbor_x, neighbor_y))
                    to_explore.append((neighbor_x, neighbor_y))

        return list(connected_coordinates)

    def save_coordinates(self):
        xml_file_path = "coordinates.xml"
        root = ET.Element("coordinates")

        # Itera a través de los objetos almacenados en el diccionario
        for object_id, coordinates in self.objects.items():
            object_elem = ET.SubElement(root, "object")
            object_elem.set("id", object_id)  # Establece el identificador único

            for coord in coordinates:
                point = ET.SubElement(object_elem, "point")
                x_elem = ET.SubElement(point, "x")
                x_elem.text = str(coord[0])
                y_elem = ET.SubElement(point, "y")
                y_elem.text = str(coord[1])
                # Añade información de color si lo deseas
                r_elem = ET.SubElement(point, "r")
                r_elem.text = str(coord[2])
                g_elem = ET.SubElement(point, "g")
                g_elem.text = str(coord[3])
                b_elem = ET.SubElement(point, "b")
                b_elem.text = str(coord[4])

        tree = ET.ElementTree(root)
        tree.write(xml_file_path)

if __name__ == "__main__":
    root = tk.Tk()
    app = App(root)
    root.mainloop()

Coordenadas del clic: 1030 297
Coordenadas del clic: 1046 347
Coordenadas del clic: 1087 489
Coordenadas del clic: 1106 534
Coordenadas del clic: 1084 558
Coordenadas del clic: 1155 695
Coordenadas del clic: 1035 807
Coordenadas del clic: 1112 828
Coordenadas del clic: 630 640
Coordenadas del clic: 684 571
Coordenadas del clic: 404 423
Coordenadas del clic: 342 533
Coordenadas del clic: 292 497
Coordenadas del clic: 287 640
